In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/GomokuAI

/content/drive/MyDrive/GomokuAI


# Git

In [ ]:
!git fetch

In [ ]:
!git pull

In [ ]:
!git add .

In [ ]:
!git config --global user.email kimjh1101@snu.ac.kr 
!git config --global user.name onetwotr1

In [ ]:
!git commit -m "minor revision"

In [ ]:
!git push origin main

# Train

In [ ]:
import importlib
import train, utils
importlib.reload(train)
importlib.reload(utils)

<module 'utils' from '/content/drive/MyDrive/GomokuAI/utils.py'>

alphazero 0 -> 750번 자체 대국 12시간

alphazero 750 -> 500번 자체 대국 8시간

alphazero 1250 -> 500번 자체 대국 20시간 (rollout 400)

alphazero 1750

In [ ]:
!python train.py -m "models/alphazero 2250.pt" -save "models/l2 0.001" -d "experience/alphazero 2250 self-play 375x8.pickle" -n 1000 -lr 0.001 --lr-decay 0.95 --early-stop 10

alphazero 0 80.pt를 선택! -> alphazero 750
750 vs 0 => 9:1

c=1.0 vs c=2.0 => 12:2 (p-value 0.012)

c=1.0 vs c=1.5 => 10:0

weight2 epoch 109 낙찰 -> alphazero 1250.pt


alphazero 1250 vs alphazero 750 => 20 : 0

alphazero 1250
rollout 400 vs 100 => 10:0

1750 : 1250 => 10:0  (아래의 1750-1) 을 사용)

alphazero 1750 후보

1) non weighted loss, no l2 constraint. epoch 60   (overfitting 되기 전이라 판단한 지점에서 끊음)

2) non weighted loss, l2 constraint (0.001) epoch 102  (맨 끝 epoch)

왠진 모르겠는데 l2 사용한 게 조금 더 빠르게 loss 떨어짐. policy, value 모두.



후보 1), 2) 비교 결과 2)가 3:17로 압승. value function overfitting이 우려됐으나 조금도 우려하지 않아도 되는 듯함

alphazero 2000 100 vs alphazero 2000 => 9 : 1
p-value 0.021484

# 성능 비교

In [ ]:
import importlib
import compare_performance, bot_v_bot, human_v_bot, utils
importlib.reload(compare_performance)
importlib.reload(bot_v_bot)
importlib.reload(human_v_bot)
importlib.reload(utils)

<module 'utils' from '/content/drive/MyDrive/GomokuAI/utils.py'>

In [ ]:
import importlib
import agent.alphazero_agent
importlib.reload(agent.alphazero_agent)

<module 'agent.alphazero_agent' from '/content/drive/MyDrive/GomokuAI/agent/alphazero_agent.py'>

In [ ]:
!python bot_v_bot.py

In [ ]:
!python compare_performance.py -m1 "models/alphazero 2250 91.pt" -m2 "models/alphazero 2250.pt" -n 10 --verbose 4 --use-model-name

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
A7     visit  11  p 0.088  v -0.78  exp_v  0.36  loss 0
G0     visit   9  p 0.014  v -0.64  exp_v  0.62  loss 0
G4     visit   9  p 0.018  v -0.73  exp_v  0.55  loss 0
A8     visit   9  p 0.050  v -0.67  exp_v  0.48  loss 0
G1     visit   7  p 0.024  v -0.67  exp_v  0.48  loss 0
B4     visit   6  p 0.023  v -0.68  exp_v  0.55  loss 0
average depth: 2.42, max depth: 5
----------------------------
Player.white (alphazero 2250) G3
 8         ●        
 7   ●    ○        
 6 ○  ●○○○○●  
 5   ●    ○        
 4     ●●○        
 3     ○●●●○    
 2         ●        
 1           ○      
 0                   
   A B C D E F G H J
G4     visit  94  p 0.067  v -0.19  exp_v  0.14  loss 15
D8     visit  63  p 0.020  v -0.37  exp_v  0.16  loss 5
A4     visit  38  p 0.020  v  0.43  exp_v  0.26  loss 0
C7     visit  22  p 0.023  v  0.36  exp_v  0.17  loss 0
D5     visit  16  p 0.084  v  0.04  exp_v -0.08  loss 0
B8     visit  15  p 0.005  v  0.36  exp_v  0.17  loss 

In [ ]:
!python compare_performance.py -m1 "models/alphazero 2250 91.pt" -m2 "models/alphazero 2250.pt" -n 10 --verbose 4 --use-model-name

  0% 0/10 [00:00<?, ?it/s]

////////  1th Game  ////////
Black: alphazero 2250 91, White: alphazero 2250
----------------------------
 8                   
 7                   
 6                   
 5                   
 4                   
 3                   
 2                   
 1                   
 0                   
   A B C D E F G H J
E2     visit 107  p 0.095  v -0.17  exp_v  0.22  loss 0
E6     visit  51  p 0.099  v -0.15  exp_v  0.17  loss 0
G4     visit  50  p 0.094  v -0.17  exp_v  0.17  loss 0
C4     visit  46  p 0.094  v -0.15  exp_v  0.16  loss 0
D3     visit  12  p 0.030  v -0.06  exp_v  0.13  loss 0
D5     visit  12  p 0.030  v  0.02  exp_v  0.14  loss 0
F5     visit  11  p 0.029  v -0.04  exp_v  0.13  loss 0
E4     visit  10  p 0.024  v -0.14  exp_v  0.14  loss 0
F3     visit   9  p 0.029  v -0.11  exp_v  0.10  loss 0
E5     visit   7  p 0.006  v -0.19  exp_v  0.19  loss 0
average depth: 2.08, max depth: 4
----------------------------
Player.black (alphazero 

# self play

In [ ]:
import importlib
import bot_v_bot, utils, alphazero_agent, compare_performance, self_play, board
# import net.alphazero_net
# importlib.reload(bot_v_bot)
# importlib.reload(compare_performance)
importlib.reload(utils)
importlib.reload(self_play)
importlib.reload(board)
importlib.reload(alphazero_agent)
# importlib.reload(net.alphazero_net)

<module 'alphazero_agent' from '/content/drive/MyDrive/GomokuAI/alphazero_agent.py'>

In [ ]:
!python self_play.py -n 1 -m "models/alphazero 2000 91 new.pt" --file-num 2 -rollout 400 -c 2 --noise-intensity 0.25 --alpha 0.1 --verbose 4

In [ ]:
for i in range(29,125):
  print()
  print(i, 'th simulation')
  !python self_play.py -n 3 -m "models/alphazero 2250 91.pt" --file-num {i} -rollout 400 -c 2.5 --noise-intensity 0.25 --alpha 0.1 --verbose 4

# 데이터 합치기

In [ ]:
import os
_list = os.listdir('experience')

In [ ]:
from pprint import pprint
files = [i for i in _list if '2000 91' in i and '.pickle' in i]
print(len(files))
pprint(files)

In [ ]:
# 개수 안 맞는 거 있나 확인
for f in files:
  exp = ExperienceCollector()
  exp.load_experience('experience/' + f)
  if len(exp.states)!=len(exp.rewards):
    print(f)
    print(len(exp))
    print(len(exp.rewards))
    print()

In [ ]:
import importlib
import experience
importlib.reload(experience)

<module 'experience' from '/content/drive/MyDrive/GomokuAI/experience.py'>

In [ ]:
from experience import *

combine_saved_experiences(['experience/' + f for f in files], "experience/alphazero 2250 self-play 220.pickle")

In [ ]:
from experience import *

combine_saved_experiences(['experience/alphazero 2250 self-play 155.pickle', 'experience/alphazero 2250 self-play 220.pickle'], "experience/alphazero 2250 self-play 375.pickle")

experience/alphazero 2250 self-play 155.pickle
3370
experience/alphazero 2250 self-play 220.pickle
5229
8599 experiences combined and saved


/content/drive/MyDrive/GomokuAI/experience.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mcts_probs_list.append(collector.mcts_probs)


txt

In [ ]:
def seconds_to_hms(seconds):
    hours = seconds // 3600
    remaining_seconds = seconds % 3600
    minutes = remaining_seconds // 60
    remaining_seconds = remaining_seconds % 60

    return hours, minutes, remaining_seconds

In [ ]:
txt_files = [i for i in _list if '2000 91' in i and '.txt' in i]
print(len(txt_files))
for f in txt_files:
    print('experience/' +f)

In [ ]:
sum_time = 0
for f in txt_files:
  file = open('experience/' + f, "r")
  lines = file.read()
  time = lines.split('\n')[-1].split(' ')[-1][:-1]
  sum_time += int(time)
  file.close()
h,m,s = seconds_to_hms(sum_time)
print(h,m,s)

13 39 26


In [ ]:
contents = "\n".join(lines.split('\n')[:-1]) + "\ntime elapsed %dh%dm%ds" %(h,m,s)
print(contents)

rollout per move 400
c 2.500000
dirichlet noise intensity 0.250000
dirichlet alpha 0.100000
time elapsed 13h39m26s


In [ ]:
file = open("experience/alphazero 2250 self-play 220.txt", "w")
contents = "\n".join(lines.split('\n')[:-1]) + "\ntime elapsed %dh%dm%ds" %(h,m,s)
file.write(contents)
file.close()

augmentation

In [ ]:
from experience import *

combined = ExperienceCollector()
combined.load_experience("experience/alphazero 2250 self-play 375.pickle")
augmented = augmentation(combined, 9)
print(len(augmented))
print(len(augmented.mcts_probs))
print(len(augmented.rewards))

68792
68792
68792


In [ ]:
augmented.save_experience("experience/alphazero 2250 self-play 375x8.pickle")

## 쪼개기, 리워드 수정

In [ ]:
def start_end(_list):
  start = [0]
  end = []
  for i in range(len(_list)):
    if _list[i]==1 or _list[i]==-1:
      end.append(i)
      start.append(i+1)
  start.pop()
  return start, end

def start_end_lose(_list):
  start = [0] if _list[0] < 0 else []
  end = []
  for i in range(len(_list)):
    if _list[i]==-1:
      end.append(i)
    if i>0 and (_list[i] < 0 and (_list[i-1]==1 or _list[i-1]==-1)):
      start.append(i)
  return start, end

In [ ]:
from experience import *
exp = ExperienceCollector()
exp.load_experience("experience/alphazero 2250 self-play 155.pickle")
start, end = start_end(exp.rewards)
print(start)
print(end)
print(len(start))
print(len(end))

In [ ]:
exp.rewards[:50]

In [ ]:
start_lose, end_lose = start_end_lose(exp.rewards)
print(start_lose)
print(end_lose)
print(len(start_lose))
print(len(end_lose))

In [ ]:
import torch

for i in range(len(start_lose)):
  s = start_lose[i]
  e = end_lose[i]
  print(exp.rewards[s:e+1])
  exp.rewards[s:e+1] = torch.tensor([-1 * 0.95 * (0.9025 ** i) for i in range(e-s+1-1, -1, -1)])
  print(exp.rewards[s:e+1])
  print()

In [ ]:
exp.save_experience("experience/alphazero 2250 self-play 155.pickle")

# Play

In [ ]:
import importlib
import human_v_bot
importlib.reload(human_v_bot)

<module 'human_v_bot' from '/content/drive/MyDrive/GomokuAI/human_v_bot.py'>

In [ ]:
!python human_v_bot.py --verbose 0